# churn predection

### import packages

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

### import data sets

In [ ]:
train = pd.read_csv('../input/TRAIN.csv')
test  = pd.read_csv('../input/TEST.csv')

### taking backup of data

In [ ]:
train_orginal = train.copy()
test_orginal  = test.copy()

In [ ]:
train.shape

In [ ]:
train.columns

In [ ]:
train.dtypes

In [ ]:
train.shape


In [ ]:
test.shape

In [ ]:
test.columns

In [ ]:
test.dtypes

In [ ]:
train.head()

#### finding catogarial variables

In [ ]:
train["Network type subscription in Month 1"].value_counts()

In [ ]:
train["Network type subscription in Month 2"].value_counts()

In [ ]:
train["Most Loved Competitor network in in Month 1"].value_counts()

In [ ]:
train["Most Loved Competitor network in in Month 2"].value_counts()

In [ ]:
train["Churn Status"].value_counts()

### type of network preffered and churn rate

In [ ]:
Tech=pd.crosstab(train['Network type subscription in Month 1'],train['Churn Status'])
Tech.div(Tech.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True, figsize=(4,4))

In [ ]:
Tech=pd.crosstab(train['Network type subscription in Month 2'],train['Churn Status'])
Tech.div(Tech.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True, figsize=(4,4))

In [ ]:
plt.figure(1)
plt.subplot(131)
train['Most Loved Competitor network in in Month 1'].value_counts().plot.bar(figsize=(24,6), title= 'Competitor_at_stage1')

plt.subplot(132)
train['Most Loved Competitor network in in Month 2'].value_counts().plot.bar(title= 'Competitor_at_stage2')

plt.show()

In [ ]:
train["Churn Status"].value_counts().plot.bar(title= 'Churn rate')

In [ ]:
train["Total Call centre complaint calls"].value_counts().plot.bar(title= 'Churn rate')


In [ ]:
# Print correlation matrix
matrix = train.corr()
f, ax = plt.subplots(figsize=(9, 6))
sns.heatmap(matrix, vmax=.8, square=True, cmap="BuPu");

##### co-related varibles are removed for better predection

In [ ]:
train=train.drop('Customer tenure in month',axis=1)

In [ ]:
train.head()

In [ ]:
train=train.drop('Total Spend in Months 1 and 2 of 2017',axis=1)

In [ ]:
train.head()

In [ ]:
train.shape

##### droping nan rows 

In [ ]:
train=train.dropna(how='all')

In [ ]:
train.isnull().sum()

##### filling missed values

In [ ]:
train['Network type subscription in Month 1'].fillna(train['Network type subscription in Month 1'].mode()[0], inplace=True)
train['Network type subscription in Month 2'].fillna(train['Network type subscription in Month 2'].mode()[0], inplace=True)
train['Most Loved Competitor network in in Month 1'].fillna(train['Most Loved Competitor network in in Month 1'].mode()[0], inplace=True)
train['Most Loved Competitor network in in Month 2'].fillna(train['Most Loved Competitor network in in Month 2'].mode()[0], inplace=True)


In [ ]:
train.isnull().sum()

In [ ]:
train=train.drop('Customer ID',axis=1)


In [ ]:
train.shape

In [ ]:
test.isnull().sum()

In [ ]:
test['Network type subscription in Month 1'].fillna(test['Network type subscription in Month 1'].mode()[0], inplace=True)
test['Network type subscription in Month 2'].fillna(test['Network type subscription in Month 2'].mode()[0], inplace=True)

#### spliting data features and target variable

In [ ]:
X = train.drop('Churn Status',1)
y = train["Churn Status"]

#### Adding dummies to the dataset

In [ ]:

X=pd.get_dummies(X)

#### selecting of model and training

In [ ]:
# Importing packages for cross validation and logistic regression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X, y)

In [ ]:
y_pred = classifier.predict(X)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, y_pred)

In [ ]:
cm

In [ ]:
y_pred

In [ ]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 10)
classifier.fit(X, y)


In [ ]:
y_pred = classifier.predict(X)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, y_pred)

In [ ]:
cm

### after applying logistic regression and DecisionTree error rate and accurcy rate are very low so applyed RandomForest is some what better

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X, y)

In [ ]:
y_pred = classifier.predict(X)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, y_pred)

In [ ]:
cm

In [ ]:
y_pred

In [ ]:
submission = test['Customer ID']

In [ ]:
test = test.drop('Customer ID',1)

In [ ]:
# Adding dummies to the dataset
test_pred=pd.get_dummies(test)

In [ ]:
test_pred=test_pred.drop('Total Spend in Months 1 and 2 of 2017',axis=1)
test_pred=test_pred.drop('Customer tenure in month',axis=1)

In [ ]:
chur_pred = classifier.predict(test_pred)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, y_pred)

In [ ]:
cm

In [ ]:
submission_result=pd.read_csv("../input/sample submission.csv")

submission_result['Customer ID']=submission
submission_result['Churn Status']=chur_pred


In [ ]:

submission_result['Churn Status'] = submission_result['Churn Status'].astype(int)


In [ ]:
submission_result.to_csv("../input/sample submission.csv",index=False)